# Lab 5 for Big Data programming.
# Apache Spark Machine Learning using Dataframes in Google Colab




# 1.	Setup an Apache Spark instance in Google Colab

In [ ]:
# Run once.

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!pip install -q findspark

#Run Once
import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"
import findspark
findspark.init()


# 2.	Create a Spark session

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark


# 3.	Download the Iris dataset and another dataset of your choosing



In [ ]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data" -O sample_data/iris.data

# 4.	Import the Iris dataset into a dataframe and insert screenshot of df.show()command output:

In [1]:
#df = spark.read.csv('sample_data/iris.data', header=False, sep=",", inferSchema=True)
df = spark.read.csv('sample_data/iris.data', inferSchema=True)\
.toDF("SepalLength","SepalWidth","PetalLength","PetalWidth","Class")

NameError: name 'spark' is not defined

# 5.	Spark ML can only deal with one features column - so we need to vectorise the multiple columns into one:

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [ ]:
vector_assembler=VectorAssembler(\
inputCols=["SepalLength","SepalWidth","PetalLength","PetalWidth"],\
outputCol="features")
df_temp=vector_assembler.transform(df)
df_temp.show(3)

In [ ]:
df_temp=df_temp.drop("SepalLength","SepalWidth","PetalLength","PetalWidth")
df_temp.show(3)


# 6.	The final data preparation step is to index the Class column - to use numeric rather than text values - run the following command and display your output of Class, features & ClassIndex columns:

In [ ]:
from pyspark.ml.feature import StringIndexer
l_indexer=StringIndexer(inputCol="Class", outputCol="ClassIndex")
df = l_indexer.fit(df).transform(df)

In [ ]:
df.show(10)

# 7.	Split your data into training and test datasets:

In [ ]:
(trainingData,testData) = df.randomSplit([0.7,0.3])

# 8.	Decision Tree Classifier 
## Specify the DecisionTreeClassifier and train the model on your training dataset:


In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier 
from pyspark.ml.evaluation importMulticlassClassificationEvaluator


In [ ]:
dt = DecisionTreeClassifier(labelCol="ClassIndex",featuresCol="features")
model=dt.fit(trainingData)

# 9.	Test your model with your test dataset: 

In [ ]:
predictions = model.transform(testData)

In [ ]:
predictions.select("prediction","ClassIndex").show(5)

# 10.	Run an evaluator function to show the accuracy of your model:

In [ ]:
evaluator= MulticlassClassificationEvaluator(\
labelCol="ClassIndex", predictionCol="prediction",\
metricName="accuracy")
accuracy=evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))
print("Test Set accuracy = " +str(accuracy))

# 11.	Random Forest Classifier

## Specify the RandomForestClassifier, train the model on your training dataset, predict using your test dataset, and run an evaluator to test accuracy:


In [ ]:
from pyspark.ml.classification import RandomForestClassifier 
rf=RandomForestClassifier(labelCol="ClassIndex",\
featuresCol="features",numTrees=10)
model=rf.fit(trainingData)
predictions=model.transform(testData)
predictions.select("prediction","ClassIndex").show(5)

In [ ]:
evaluator= \
MulticlassClassificationEvaluator(labelCol="ClassIndex",\
predictionCol="prediction",metricName="accuracy")
accuracy=evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))
print("Test Set accuracy = " +str(accuracy))

# 12.	Naive Bayes Classifier
## Specify the NaiveBayes classifier, train the model on your training dataset, predict using your test dataset, and run an evaluator to test accuracy:


In [ ]:
from pyspark.ml.classification import NaiveBayes 
nb=NaiveBayes(labelCol="ClassIndex",\
featuresCol="features",smoothing=1.0,\
modelType="multinomial")
model=nb.fit(train)


In [ ]:
predictions=model.transform(test)
predictions.select("Class","ClassIndex",
"probability","prediction").show(5)

In [ ]:
evaluator= \
MulticlassClassificationEvaluator(labelCol="ClassIndex",\
predictionCol="prediction",metricName="accuracy")
accuracy=evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))
print("Test Set accuracy = " +str(accuracy))